<br>
<h1 style="font-family:sans-serif; text-align:center"> 
<!--     <span style='color: pink'> Twitter analysis of </span> -->
    <span style='color: white; font-size:100%; text-shadow: 0px 0px 15px black'> Twitter analysis of </span>
<!--     <span style='color:#00acee'> Twitter analysis of </span> -->
<!--     <span style="-webkit-text-stroke"> Twitter analysis of</span> -->
<!--     <span class="hr3" style='color:#e40843; letter-spacing: 4px; font-size:105%'> Canada</span> -->
    <span class="hr3" style='color:#e40843; font-size:120%; text-shadow: 0px 0px 30px pink'>Canada </span> <span class="hr3" style='color:gray; font-size:100%; text-shadow: 0px 0px 30px pink'>response to Covid-19</span><br>
</h1>

#### — _Using snscrape_ —

### ✅ This jupyter notebook works well!

The aim of this notebook is to retrieve the tweets from March 1st until April 30th, to analyze the difference in sentiment analysis of tweets from people before and after Trudeau's [announcement of government policies facing impact of Covid-19](https://www.youtube.com/watch?v=1o-tV0A87l8&feature=youtu.be) to support small businesses and their employees.


The **snscrape** allowed us to find old tweets (as opposed to the free version of the API from twitter, and the GetOldTweets3 library that is non-currently working). The way to download tweets with this library is well explained in Martin Beck's article _[How to Scrape Tweets With snscrape](https://medium.com/better-programming/how-to-scrape-tweets-with-snscrape-90124ed006af)_ at Medium.

_Authors: Leo Cuspinera ([cuspime](https://github.com/cuspime)) and Victor Cuspinera ([vcuspinera](https://github.com/vcuspinera))_

## Imports

In [12]:
# # Install development version of snscrape
# !pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git 

# General libraries
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime, timedelta, date
from pytz import timezone
import json

# Preprocess libraries
import re
import spacy
import string
import en_core_web_sm
nlp = en_core_web_sm.load()

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Parameters

In [13]:
START = time.time()

# dates
today = datetime.now()
init = date.fromisoformat('2020-02-01')

my_dates = list()
for d in range(0, 90, 1):
    aux = init + timedelta(days=d)
    my_dates.append(aux)

# twitter accounts
accounts = ("@JustinTrudeau", "@CanadianPM","@Canada", "@GovCanHealth")

# max number of results
max_results = 100_000

#folder to save information
my_folder = "../tweets/"

## Get and save tweets as `json` files
⚠️ **Caution:** Just run this code chunk once, it takes so much time (more than couple hours) to download all the tweets. Additionally, this step downloads 244 JSON files, that in total weight 10.27 GB

In [ ]:
%%time

# Retrieving tweets with `snscrape`, by using OS library to call CLI commands in Python.
for ac in accounts:
    for dt in my_dates:
        next_day = dt + timedelta(days=1)
        os.system("snscrape --jsonl --max-results " + str(max_results) + " --since " + 
                  dt.strftime("%Y-%m-%d") + " twitter-search '" + ac + " until:" + 
                  next_day.strftime("%Y-%m-%d") + "' > " + my_folder + ac + 
                  "_" + dt.strftime("%Y-%m-%d") + ".json")


## Bring and merge tweets by account

In [14]:
%%time

# Columns
my_columns = ['account', 'date', 'content', 'user', 'replyCount', 'retweetCount',
              'likeCount', 'quoteCount', 'lang', 'sourceLabel']

dict_tot = {}

# # Call and concatenate the data frames
for ac in accounts:
    # Create an empty pandas dataframe
    df = pd.DataFrame(columns = my_columns)
    t0 = time.clock()
    print("Start with " + ac + " files:")

    # Call the JSON files of tweets
    for d in my_dates:
        t00 = time.clock()
        aux = pd.read_json(my_folder + ac + '_' + str(d) + '.json', lines=True)
        aux['account'] = ac
        df = pd.concat([df, aux[my_columns]])
        print("   > date: " + str(d) + ' time ' + str(round(time.clock() - t00, 4)))

    # Save a JSON file for each account
    df.reset_index(drop=True, inplace=True)
    dict_tot[ac] = df
    print(ac + " DB ready, time " + str(round(time.clock() - t0, 4)))
    # df.to_csv(my_folder + 'tweets_db_' + ac + '.csv')
    print(ac + " tweets saved as .JSON file, time " + str(round(time.clock() - t0, 4)))
    print("- - - - - o - - - - -\n")


Start with @JustinTrudeau files:
   > date: 2020-02-01 time 0.3718
   > date: 2020-02-02 time 0.3239
   > date: 2020-02-03 time 0.2696
   > date: 2020-02-04 time 0.199
   > date: 2020-02-05 time 0.4545
   > date: 2020-02-06 time 0.3254
   > date: 2020-02-07 time 0.4981
   > date: 2020-02-08 time 0.4872
   > date: 2020-02-09 time 0.7022
   > date: 2020-02-10 time 0.6538
   > date: 2020-02-11 time 0.3519
   > date: 2020-02-12 time 0.6593
   > date: 2020-02-13 time 0.7202
   > date: 2020-02-14 time 0.8659
   > date: 2020-02-15 time 0.7752
   > date: 2020-02-16 time 0.5921
   > date: 2020-02-17 time 0.6629
   > date: 2020-02-18 time 1.1729
   > date: 2020-02-19 time 1.2126
   > date: 2020-02-20 time 0.8324
   > date: 2020-02-21 time 0.5746
   > date: 2020-02-22 time 0.4616
   > date: 2020-02-23 time 0.6444
   > date: 2020-02-24 time 0.7931
   > date: 2020-02-25 time 0.7965
   > date: 2020-02-26 time 0.6174
   > date: 2020-02-27 time 1.1044
   > date: 2020-02-28 time 0.5402
   > date: 2020-

## Merge all tweets in one `json` file

⚠️ Just run this section once: the first time you run the notebook. If you previously run this cell and you are reopening this notebook, go to the next section _**Open Json file with all tweets**_.

In [15]:
%%time
df_tot = pd.DataFrame()
# Concatenate all tweets
df_tot = pd.concat([dict_tot['@Canada'], 
                    dict_tot['@CanadianPM'], 
                    dict_tot['@GovCanHealth'], 
                    dict_tot['@JustinTrudeau']])
df_tot.reset_index(drop=True, inplace=True)

CPU times: user 883 ms, sys: 685 ms, total: 1.57 s
Wall time: 2.06 s


In [16]:
def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
        del ret[column]
    return ret

In [17]:
%%time

# unpack information from user
df_tot = unpack(df_tot, 'user')

# select only the columns we wanted to save
wanted_columns = ['account', 'date', 'content', 'replyCount', 'retweetCount', 
                  'likeCount', 'quoteCount', 'lang', 'sourceLabel', 'username', 
                  'followersCount', 'friendsCount', 'location']
df_tot = df_tot[wanted_columns]

CPU times: user 11.7 s, sys: 463 ms, total: 12.1 s
Wall time: 12.1 s


In [18]:
%%time

# Save tweets as JSON file
df_tot.to_json(my_folder + 'tweets_db.json')

CPU times: user 9.54 s, sys: 1.87 s, total: 11.4 s
Wall time: 14.1 s


## Open `json` file with all tweets
#### 👉 Use this in case you already run the previous chunks, and you are reopening the notebook.

In [19]:
%%time

# Open the file
df_tot = pd.read_json(my_folder + 'tweets_db.json')

CPU times: user 11.8 s, sys: 2.88 s, total: 14.7 s
Wall time: 15.7 s


In [20]:
print("Total time to run this notebook (until this point):", np.round((time.time() - START)/60, 2), "minutes.")

Total time to run this notebook (until this point): 3.0 minutes.


## Preprocess tweets
#### ⚠️ This process is very slow because it changes more than 70 thousand tweets, one by one. 
Instead of running this section, we recommend to use the python script __[preprocess.py](https://github.com/vcuspinera/Canada_response_covid/blob/master/src/preprocess.py)__ by running from the terminal the next code:  
`$ python src/preprocess.py --input_dir=tweets/ --output_dir=tweets/`

In [21]:
def preprocess(text, irrelevant_pos = ['SPACE'],
              avoid_entities = ['ORG']):
    """
    Function that identify sensible information and delete some of 
    these data as emails and urls.
    Parameters
    -------------
    text : (list)
        the list of text to be preprocessed
    irrelevant_pos : (list)
        a list of irrelevant 'pos' tags
    avoid_entities : (list)
        a list of entity labels to be avoided

    Returns
    -------------
    (list) list of preprocessed text

    Example
    -------------
    example = ["Contact me at george23@gmail.com",
           "@vcuspinera my webpage is https://vcuspinera.github.io"]
    preprocess(example)
    (output:) ['contact me at',
               'my webpage is']
    """
    result = []

    # function
    for sent in text:
        sent = str(sent).lower()

        result_sent = []
        doc = nlp(sent)
        entities = [str(ent) for ent in doc.ents if ent.label_ in avoid_entities]
        # This helps to detect names organization

        for token in doc:            
            if (token.like_email or
                token.like_url or
                token.pos_ in irrelevant_pos or
                str(token) in entities
               ):
                continue
            else:
                if str(token) in string.punctuation:
                    try:
                        result_sent[-1] = str(result_sent[-1]) + str(token)
                    except:
                        result_sent.append(str(token))
                else:
                    result_sent.append(str(token))
        result.append(" ".join(result_sent))
    return result

In [ ]:
# Preprocess tweets
df_tot['tweet'] = preprocess(df_tot['content'])

In [ ]:
# Save file
df_tot.drop('content', axis=1).to_json(output_dir + 'tweets_db_clean.json')

## **Basic analysis** and **EDA**

#### 👉 [Click here](https://github.com/vcuspinera/Canada_response_covid/blob/master/src/eda.ipynb) to see the initial Data Analysis in the EDA jupyter notebook of this project.